## Imports

In [1]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 17.4 MB/s 
     |████████████████████████████████| 6.6 MB 47.5 MB/s 
     |████████████████████████████████| 77 kB 4.5 MB/s 
     |████████████████████████████████| 895 kB 82.9 MB/s 
     |████████████████████████████████| 596 kB 51.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import os
from tqdm.notebook import tqdm
import glob
import sys
import time
import pickle
import cv2
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import random
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import get_scheduler

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

## Load DataSet

In [4]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [5]:
%cd /content/drive/MyDrive/DL_Project

/content/drive/.shortcut-targets-by-id/1mJgNU1aJtlW2TXPFO2e4TWL1mJ2AVebd/DL_Project


In [6]:
BASE_DIR = './newestDataset/'

In [ ]:
# df = pd.read_csv('./Num.csv', dtype=str)
# df[df['text'].str.len()==5]
# # df[df['file']=='Num.csv']
# # df = pd.concat([df.iloc[:999] , df.iloc[1000:]])
# # df.to_csv('./NewDataset/Num2.csv', index=False)

In [7]:
class CaptchaDataset(Dataset):
    def __init__(self, root_dir, annotation_file, transform=None):
        self.root_dir = root_dir
        self.annotations = pd.read_csv(f"{root_dir}{annotation_file}", dtype=str)
        self.annotations = self.annotations[self.annotations['length']=='5']
        self.annotations = self.annotations
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_id = self.annotations.iloc[index]
        img = Image.open(
            # os.path.join( self.root_dir, 'NewDataset', 'data', img_id['category'], f"Len{img_id['length']}Char", img_id['file'])
            os.path.join( self.root_dir, 'NewDataset', f"Len{img_id['length']}Char", 'data', img_id['file'])
          ).convert("L")
        # print(img_id['length'], len(str(img_id['text'])))
        y_label = (int(img_id['length']), img_id['text']) #int(img_id['text'][1]), int(img_id['text'][2]), int(img_id['text'][3]), int(img_id['text'][4]))

        if self.transform is not None:
            img = self.transform(img)

        return (img, y_label)
# class CaptchaDataset(Dataset):
#     def __init__(self, root_dir, annotation_file, transform=None):
#         self.root_dir = root_dir
#         self.annotations = pd.read_csv(f"./{annotation_file}", dtype=str)
#         self.transform = transform

#     def __len__(self):
#         return len(self.annotations)

#     def __getitem__(self, index):
#         img_id = self.annotations.iloc[index]
#         img = Image.open(
#             # os.path.join( self.root_dir, img_id['category'], f"Len{img_id['length']}Char", img_id['file'])
#             os.path.join( self.root_dir, img_id['file'])
#           ).convert("L")
#         # print('qwerty  ', img_id['text'])
#         y_label = (img_id['length'], str(img_id['text']))

#         if self.transform is not None:
#             img = self.transform(img)
  
#         return (img, y_label)

In [8]:
transform = transforms.Compose(
    [
        # transforms.Resize((135//2, 50//2)),
        # transforms.RandomCrop((299, 299)),
        # transforms.Resize((256,256)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, ), (0.5, )),
    ]
)

In [9]:
BATCH = 16

In [12]:
dataset = CaptchaDataset(BASE_DIR, "Num.csv", transform = transform)
print(len(dataset))
train_set, validation_set, test_set = torch.utils.data.random_split(dataset, [4000, 500, 500])
# train_set, validation_set = torch.utils.data.random_split(dataset, [8000,2001])

train_loader = DataLoader(dataset = train_set, shuffle = True, batch_size = BATCH)
validation_loader = DataLoader(dataset = validation_set, shuffle = True, batch_size = BATCH)
test_loader = DataLoader(dataset = test_set, shuffle = True, batch_size = BATCH)

5000


In [13]:
dataiter = iter(train_loader)
images, labels = dataiter.next()
print(type(images))
print(images.shape)
print(type(labels[1]))

<class 'torch.Tensor'>
torch.Size([16, 1, 60, 200])
<class 'tuple'>


In [ ]:
# print(labels[1])

In [ ]:
# for i, data in enumerate(train_loader):
#     inputs = data[0].to(device)
#     labels = data[1]

#     # length = data[1][0][0]
#     # t = torch.zeros(1, 5).to(device)
#     # t[length-1] = 1
#     # length = t

#     label = []
    
#     for l in range(len(labels[0])):
#         label.append([])
#         length = labels[0][l]

#         for j in range(int(length)):
#             print(labels[1][l])
#             character = labels[1][l][j]
#             output_array = [0]*11
#             output_array[int(character)] = 1
#             label[l].append(output_array)

#         for j in range(int(length), 5):
#             output_array = [0]*11
#             output_array[10] = 1
#             label[l].append(output_array)


#     label = torch.Tensor(label).to(device)


In [ ]:
# torch.stack([outputs['label1'], outputs['label2']], axis=1)

In [ ]:
# print(labels)

In [14]:
DROPOUT_PROB = 0.3
# N_EPOCHS = 10
# model_path = './Models/'

In [15]:
class Model(nn.Module):
    def __init__(self, dropout_flag=1):
        super().__init__()

        # self.dropout_flag = dropout_flag  # 1->Conv Layer, 2->FC Layer

        self.blockA = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5, stride=1, padding='same'),
            nn.ReLU(),
        )

        self.blockA_drop = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5, stride=1, padding='same'),
            nn.ReLU(),
            nn.Dropout(DROPOUT_PROB),
        )

        self.blockB = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=48, kernel_size=5, stride=1, padding='same'),
            nn.ReLU(),
        )

        # self.blockB_drop = nn.Sequential(
        #     nn.Conv2d(in_channels=32, out_channels=48, kernel_size=5, stride=1, padding='same'),
        #     nn.ReLU(),
        #     nn.Dropout(DROPOUT_PROB),
        # )

        # self.blockC = nn.Sequential(
        #     nn.Conv2d(in_channels=48, out_channels=64, kernel_size=5, stride=1, padding='same'),
        #     nn.ReLU(),
        # )

        self.blockC_drop = nn.Sequential(
            nn.Conv2d(in_channels=48, out_channels=64, kernel_size=5, stride=1, padding='same'),
            nn.ReLU(),
            nn.Dropout(DROPOUT_PROB),
        )


        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(11200, 512)
        self.out1 = nn.Linear(512, 10)
        self.out2 = nn.Linear(512, 10)
        self.out3 = nn.Linear(512, 10)
        self.out4 = nn.Linear(512, 10)
        self.out5 = nn.Linear(512, 10)

        self.length_out = nn.Linear(512, 5)

        self.softmax = nn.Softmax()
        self.relu = nn.ReLU()

        # self.drop = nn.Dropout(DROPOUT_PROB)
        

    def forward(self, x):
        
        x = self.blockA(x)
        x = self.pool(x)
        
        x = self.blockB(x)
        x = self.pool(x)
        
        x = self.blockC_drop(x)
        x = self.pool(x)
        
        x = torch.flatten(x, start_dim=1)
        
        x = self.relu(self.fc1(x))

        # label1 = self.softmax(self.out1(x))
        # label2 = self.softmax(self.out2(x))
        # label3 = self.softmax(self.out3(x))
        # label4 = self.softmax(self.out4(x))
        # label5 = self.softmax(self.out5(x))

        # length = self.softmax(self.length_out(x))

        label1 = self.out1(x)
        label2 = self.out2(x)
        label3 = self.out3(x)
        label4 = self.out4(x)
        label5 = self.out5(x)
        

        return {'label1':label1, 'label2':label2, 'label3':label3, 'label4':label4, 'label5':label5}

        # return {'label': torch.stack([label1, label2, label3, label4, label5], dim=1), 'length': length}

In [20]:
model = Model().to(device)

In [21]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=5e-3)


In [22]:
## create the default learning rate scheduler
num_epochs = 50
num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_scheduler(
    name="linear", 
    optimizer=optimizer, 
    num_warmup_steps=0, 
    num_training_steps=num_training_steps
)

In [23]:
# torch.Tensor([1]).to(device)

In [24]:
from torch.nn.utils.rnn import pad_sequence
# def train(model, title, regularization = None):
    
progress_bar = tqdm(range(num_training_steps))
trainlosslist = []
validationlosslist = []

for epoch in range(num_epochs):
    train_loss = 0.0
    train_correct1 = 0
    train_correct2 = 0
    train_correct3 = 0
    train_correct4 = 0
    train_correct5 = 0
    train_correct = 0
    train_total = 0

    validation_loss = 0.0
    validation_correct = 0
    validation_total = 0

    model.train()
    for i, data in enumerate(train_loader):
        inputs = data[0].to(device)
        labels = data[1]
        
        label = []
        
        # for l in range(len(labels[0])):
        #     label.append([])
        #     length = labels[0][l]

        #     for j in range(int(length)):
        #         character = labels[1][l][j]
        #         output_array = [0]*10
        #         output_array[int(character)] = 1
        #         label[l].append(output_array)

        #     for j in range(int(length), 5):
        #         output_array = [0]*10
        #         output_array[10] = 1
        #         label[l].append(output_array)

        label1 = []
        label2 = []
        label3 = []
        label4 = []
        label5 = []
        for l in range(len(labels[0])):
            label1.append([])
            label2.append([])
            label3.append([])
            label4.append([])
            label5.append([])
            length = labels[0][l]

            char1 = labels[1][l][0]
            output_array1 = [0]*10
            output_array1[int(char1)] = 1
            label1[l].append(output_array1)

            char2 = labels[1][l][1]
            output_array2 = [0]*10
            output_array2[int(char2)] = 1
            label2[l].append(output_array2)

            char3 = labels[1][l][2]
            output_array3 = [0]*10
            output_array3[int(char3)] = 1
            label3[l].append(output_array3)

            char4 = labels[1][l][3]
            output_array4 = [0]*10
            output_array4[int(char4)] = 1
            label4[l].append(output_array4)

            char5 = labels[1][l][4]
            output_array5 = [0]*10
            output_array5[int(char5)] = 1
            label5[l].append(output_array5)

        label1 = torch.Tensor(label1).to(device)
        label2 = torch.Tensor(label2).to(device)
        label3 = torch.Tensor(label3).to(device)
        label4 = torch.Tensor(label4).to(device)
        label5 = torch.Tensor(label5).to(device)

        label1 = label1.reshape(label1.shape[0],10)
        label2 = label2.reshape(label2.shape[0],10)
        label3 = label3.reshape(label3.shape[0],10)
        label4 = label4.reshape(label4.shape[0],10)
        label5 = label5.reshape(label5.shape[0],10)


        output = model(inputs.to(device))
        out1 = output['label1']
        out2 = output['label2']
        out3 = output['label3']
        out4 = output['label4']
        out5 = output['label5']

        loss1 = criterion(out1, label1)
        loss2 = criterion(out2, label2)
        loss3 = criterion(out3, label3)
        loss4 = criterion(out4, label4)
        loss5 = criterion(out5, label5)

        loss = loss1 + loss2 + loss3 + loss4 + loss5

        # print(label1, '   lf  ', label1.shape)
        # print(out1, '   out  ', out1.shape)
        # break
        pred1 = torch.argmax(out1, 1)
        pred2 = torch.argmax(out2, 1)
        pred3 = torch.argmax(out3, 1)
        pred4 = torch.argmax(out4, 1)
        pred5 = torch.argmax(out5, 1)

        label_1 = torch.argmax(label1, 1)
        label_2 = torch.argmax(label2, 1)
        label_3 = torch.argmax(label3, 1)
        label_4 = torch.argmax(label4, 1)
        label_5 = torch.argmax(label5, 1)


        train_correct1 = (pred1==label_1).sum()
        train_correct2 = (pred2==label_2).sum()
        train_correct3 = (pred3==label_3).sum()
        train_correct4 = (pred4==label_4).sum()
        train_correct5 = (pred5==label_5).sum()

        train_total += BATCH

        # print(label1, '   LABEL  ', label1.shape)
        # print(out1, '   OUT  ', out1.shape)
        # print(pred1,'   PREDICT  ', label_1)
        # print(train_correct1, '   acc')
        # break   



        # label = torch.Tensor(label).to(device)
        # outputs = model(inputs.to(device))

        # # calculate loss
        # loss_label = criterion(outputs['label'], label)

        # # loss = loss #+ loss2 + loss3 + loss4 + loss5
        # loss = loss_label

        # pred = torch.argmax(outputs['label'], 2)
        
        # label = torch.argmax(label, axis=2)

        # train_correct += (torch.sum(label==pred, axis=1)==5).nonzero().shape[0]
        
        # train_total += BATCH
        
        # correct = torch.sum(label==pred, axis=0)
        

        # # train_correct1 += (label[0][0][pred[0][0]] == 1)
        # # train_correct2 += (label[0][1][pred[0][1]] == 1)
        # # train_correct3 += (label[0][2][pred[0][2]] == 1)
        # # train_correct4 += (label[0][3][pred[0][3]] == 1)
        # # train_correct5 += (label[0][4][pred[0][4]] == 1)

        # train_correct1 += (correct[0])
        # train_correct2 += (correct[1])
        # train_correct3 += (correct[2])
        # train_correct4 += (correct[3])
        # train_correct5 += (correct[4])

        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        progress_bar.update(1)
        # print statistics
        train_loss += loss.item()
        
        
    # for i, data in enumerate(validation_loader):
    #     model.eval()
    #     inputs, labels = data[0].to(device), data[1].to(device)
    #     labels = labels.to(torch.float32)
        
    #     outputs = model(inputs.to(device))
    #     outputs = torch.reshape(outputs, (outputs.shape[0],))
    #     loss = criterion(outputs, labels.to(device))
    #     validation_loss += loss.item()
        
        
    #     predicted = (outputs>0.5).float()
    #     validation_total += labels.size(0)
    #     validation_correct1 += (predicted == labels.to(device)).sum().item()
    #     validation_correct2 += (predicted == labels.to(device)).sum().item()
    #     validation_correct3 += (predicted == labels.to(device)).sum().item()
    #     validation_correct4 += (predicted == labels.to(device)).sum().item()
    #     validation_correct5 += (predicted == labels.to(device)).sum().item()


    epoch_train_loss = train_loss/len(train_loader)
    # epoch_validation_loss = validation_loss/len(validation_loader) 
    epoch_train_acc1 = (train_correct1/train_total)*100
    epoch_train_acc2 = (train_correct2/train_total)*100
    epoch_train_acc3 = (train_correct3/train_total)*100
    epoch_train_acc4 = (train_correct4/train_total)*100
    epoch_train_acc5 = (train_correct5/train_total)*100
    # epoch_validation_acc = (validation_correct1/validation_total)*100

    print("Epoch [%d/%d]  Training Loss: %.5f  Acc1: %.5f  Acc2: %.5f  Acc3: %.5f  Acc4: %.5f  Acc5: %.5f " %(epoch+1, num_epochs, epoch_train_loss, epoch_train_acc1, epoch_train_acc2, epoch_train_acc3, epoch_train_acc4, epoch_train_acc5))
    print(f"Epoch {epoch},  Loss : {epoch_train_loss},  Acc : {train_correct/train_total}")
    
    # print("Epoch [%d/%d]  Training Loss: %.5f  Training Acc: %.5f  Validation Loss: %.5f   Validation Accuracy: %.5f " %(epoch+1, N_EPOCHS, epoch_train_loss, epoch_train_acc, epoch_validation_loss, epoch_validation_acc))
    trainlosslist.append(epoch_train_loss)
    # validationlosslist.append(epoch_validation_loss)
    
    # if (epoch_validation_loss < min_validation_loss):
    #     min_validation_loss = epoch_validation_loss
    #     torch.save(model.state_dict(), str(model_path)+str(title)+'.pt')
    

print('Finished Training')

# plot(trainlosslist, validationlosslist, title)

  0%|          | 0/12500 [00:00<?, ?it/s]

Epoch [1/50]  Training Loss: 14.32286  Acc1: 0.17500  Acc2: 0.07500  Acc3: 0.00000  Acc4: 0.10000  Acc5: 0.02500 
Epoch 0,  Loss : 14.322860069274903,  Acc : 0.0
Epoch [2/50]  Training Loss: 10.53596  Acc1: 0.20000  Acc2: 0.05000  Acc3: 0.05000  Acc4: 0.05000  Acc5: 0.02500 
Epoch 1,  Loss : 10.535964637756347,  Acc : 0.0
Epoch [3/50]  Training Loss: 10.45356  Acc1: 0.20000  Acc2: 0.05000  Acc3: 0.00000  Acc4: 0.07500  Acc5: 0.05000 
Epoch 2,  Loss : 10.453556770324708,  Acc : 0.0
Epoch [4/50]  Training Loss: 10.45093  Acc1: 0.30000  Acc2: 0.07500  Acc3: 0.00000  Acc4: 0.07500  Acc5: 0.02500 
Epoch 3,  Loss : 10.450928539276124,  Acc : 0.0
Epoch [5/50]  Training Loss: 10.46552  Acc1: 0.22500  Acc2: 0.07500  Acc3: 0.05000  Acc4: 0.02500  Acc5: 0.02500 
Epoch 4,  Loss : 10.465523567199707,  Acc : 0.0
Epoch [6/50]  Training Loss: 10.58066  Acc1: 0.25000  Acc2: 0.02500  Acc3: 0.10000  Acc4: 0.00000  Acc5: 0.02500 
Epoch 5,  Loss : 10.58065665435791,  Acc : 0.0
Epoch [7/50]  Training Loss: 

In [ ]:
a = torch.zeros(16,5)
a[0][1]=10
a[1][3]=10
a[1][4]=10
a[2][2]=10
a

In [ ]:
# for i, data in enumerate(validation_loader):
#     model.eval()

#     inputs = data[0].to(device)
#     labels = data[1]

#     label1 = labels[1].to(device)
#     # label2 = labels[2].to(device)
#     # label3 = labels[3].to(device)
#     # label4 = labels[4].to(device)
#     # label5 = labels[5].to(device)
    

#     # forward + backward + optimize
#     outputs = model(inputs.to(device))

#     label1_hat = outputs['label1']
#     # label2_hat = outputs['label2']
#     # label3_hat = outputs['label3']         
#     # label4_hat = outputs['label4']         
#     # label5_hat = outputs['label5']  

#     _, predicted1 = torch.max(label1_hat.data, 1)
#     # _, predicted2 = torch.max(label2_hat.data, 1)
#     # _, predicted3 = torch.max(label3_hat.data, 1)
#     # _, predicted4 = torch.max(label4_hat.data, 1)
#     # _, predicted5 = torch.max(label5_hat.data, 1)

#     validation_total += label1.size(0)

#     validation_correct1 += (predicted1 == label1.to(device)).sum().item()
#     # validation_correct2 += (predicted2 == label2.to(device)).sum().item()
#     # validation_correct3 += (predicted3 == label3.to(device)).sum().item()
#     # validation_correct4 += (predicted4 == label4.to(device)).sum().item()
#     # validation_correct5 += (predicted5 == label5.to(device)).sum().item()

#     # epoch_validation_loss = validation_loss/len(validation_loader) 
#     epoch_validation_acc1 = (validation_correct1/validation_total)*100
#     # epoch_validation_acc2 = (validation_correct2/validation_total)*100
#     # epoch_validation_acc3 = (validation_correct3/validation_total)*100
#     # epoch_validation_acc4 = (validation_correct4/validation_total)*100
#     # epoch_validation_acc5 = (validation_correct5/validation_total)*100

#     epoch_validation_acc2 = 0
#     epoch_validation_acc3 = 0
#     epoch_validation_acc4 = 0
#     epoch_validation_acc5 = 0

#     print("Epoch [%d/%d]  Acc1: %.5f  Acc2: %.5f  Acc3: %.5f  Acc4: %.5f  Acc5: %.5f " %(epoch+1, num_epochs, epoch_validation_acc1, epoch_validation_acc2, epoch_validation_acc3, epoch_validation_acc4, epoch_validation_acc5))
    


AttributeError: ignored

In [ ]:
# model = Model().to(device)

In [ ]:
# title = 'yeah'
# train(model, title)

  0%|          | 0/10 [01:47<?, ?it/s]


KeyboardInterrupt: ignored

In [ ]:
# def load_data(path, C_type='/AlphaNum', char_len = 5):
#     print('loading dataset')
#     X_train = []
#     Y_train = []
#     X_test = []
#     Y_test = []

#     path = path + C_type

#     for root, dirs, files in os.walk(path):
#         for f in files:
#             if 'Len5Char' in os.path.join(root,f):
#                 if '.png' in f:
#                     fl = f.split("_")[0]
#                     print(fl)
                    

#                     label = np.zeros((char_len, 36))
#                     for i in range(char_len):
#                         label[i, int(fl[i])] = 1

#                     img = cv2.imread(os.path.join(root, fl))
#                     img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#                     img = cv2.resize(img, (int(135/2), int(50/2)), interpolation=cv2.INTER_AREA)
#                     img = np.reshape(img, (img.shape[0], img.shape[1], 1))

#                     if random() < 0.1:
#                         Y_test.append(label)
#                         X_test.append(img)
#                     else:
#                         Y_train.append(label)
#                         X_train.append(img)






## Visualize Images

In [ ]:
def imshow(img, title):
    plt.figure()
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    npimg = np.transpose(npimg, (1,2,0))
    plt.title(title)
    plt.imshow(npimg)
    plt.show()

dataiter = iter(dataloader)
images, labels = dataiter.next()

c=0
for i in range(len(images)):
    if (labels[i]==0):
        c+=1
        imshow(images[i], "Parasitized")
    if (c==5):
        break

c=0
for i in range(len(images)):
    if (labels[i]==1):
        c+=1
        imshow(images[i], "Uninfected")
    if (c==5):
        break

In [ ]:
def plot(trainlosslist, validationlosslist, title):
    epochs = range(1,N_EPOCHS+1)
    plt.plot(epochs, trainlosslist, 'g', label='Training loss')
    plt.plot(epochs, validationlosslist, 'b', label='Validation loss')
    plt.title('Training and Validation loss  - ' + str(title))
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

In [ ]:
def testing(model):
    
    model.eval()
    
    test_correct = 0
    test_total = 0

    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images.to(device))
            outputs = torch.reshape(outputs, (outputs.shape[0],))
            predicted = (outputs>0.5).float()
            test_total += labels.size(0)
            test_correct += (predicted == labels.to(device)).sum().item()
            

    print('Accuracy of the network on the test images: %.5f %%' % (100*test_correct/test_total))